In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

In [2]:
df = pd.read_excel('CPC-Algonquin College - Depot Dashboard - Data v2 - 2019-24.xlsx', sheet_name='Sheet1')

In [3]:
weather_df = pd.read_csv('climate-daily.csv')

In [4]:
attendence_df = pd.read_excel('Vancouver Plant Attendance dataset (2019-2024).xlsx', sheet_name="DATA")

In [5]:
attendence_df.head()

,Date,Activity,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences,WEEK,YEAR
0,2019-01-02,APOC,728.00,72.00,47.00,0.00,1,2019
1,2019-01-02,MP Plant,8157.90,1123.23,600.96,296.59,1,2019
2,2019-01-03,APOC,752.00,80.00,64.00,0.00,1,2019
3,2019-01-03,MP Plant,8904.08,1138.42,498.20,283.91,1,2019
4,2019-01-04,APOC,720.00,58.00,64.00,0.00,1,2019


In [6]:
attendence_df_APOC = attendence_df[attendence_df['Activity'] == 'APOC']

In [7]:
attendence_df_MP = attendence_df[attendence_df['Activity'] == 'MP Plant']

In [8]:
attendence_df_trimed = attendence_df.drop(columns=['Activity', 'WEEK','YEAR'])

In [9]:
attendence_df_trimed = attendence_df_trimed.rename(columns={'Date': 'CALENDAR_DATE'})

In [10]:
attendence_df_grouped = attendence_df_trimed.groupby('CALENDAR_DATE').sum()

In [11]:
attendence_df_grouped=attendence_df_grouped.reset_index()

In [130]:
attendence_df_grouped.head()

,CALENDAR_DATE,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences
0,2019-01-02,8885.90,1195.23,647.96,296.59
1,2019-01-03,9656.08,1218.42,562.20,283.91
2,2019-01-04,8795.96,1269.42,578.67,277.13
3,2019-01-05,4354.98,575.00,305.32,132.69
4,2019-01-06,6250.94,676.13,472.46,253.29


In [131]:
df.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,WEEK_OF_YEAR,DAY_OF_MONTH,MONTH_NO,YEAR_NO,COST_CENTRE_ID,PEAK,SITE_PROVINCE_CODE,ACTUAL_VOLUME,PLANT_NAME_EN,PCR_CC_ROLLUP,PCR_CC_CITY,PCR_CC_POSTAL_CODE
0,2019-01-01,1,0,0,3,0,1,1,2019,639524,1,BC,0,VANCOUVER MPP,LCD 3 SURREY,SURREY,V3T 2Y0
1,2019-01-01,1,0,0,3,0,1,1,2019,51217,1,BC,0,VANCOUVER MPP,Quesnel Office,QUESNEL,V2J 2M4
2,2019-01-01,1,0,0,3,0,1,1,2019,101059,1,BC,0,VANCOUVER MPP,STN KINGSTON,KINGSTON,K7M 5E7
3,2019-01-01,1,0,0,3,0,1,1,2019,646040,1,BC,0,VANCOUVER MPP,Quesnel Office,QUESNEL,V2J 2M4
4,2019-01-01,1,0,0,3,0,1,1,2019,641596,1,BC,0,VANCOUVER MPP,SHN050997 - OPS LAS COURTENAY PAC,COURTENAY,V9N 1G0


In [12]:
df_noLocation = df.drop(columns=['WEEK_OF_YEAR','DAY_OF_MONTH','MONTH_NO','YEAR_NO','COST_CENTRE_ID','SITE_PROVINCE_CODE','PLANT_NAME_EN','PCR_CC_ROLLUP','PCR_CC_CITY','PCR_CC_POSTAL_CODE'])

In [13]:
df_noLocation=df_noLocation.groupby('CALENDAR_DATE').sum().reset_index()

In [16]:
df_noLocation.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME
0,2019-01-01,43,0,0,129,43,8
1,2019-01-02,0,0,0,172,43,40804
2,2019-01-03,0,0,0,215,43,42189
3,2019-01-04,0,0,0,258,43,44384
4,2019-01-05,0,0,0,301,43,11


In [14]:
df_noLocation.loc[df_noLocation['PEAK']!=0,'PEAK']=1

In [15]:
df_noLocation.loc[df_noLocation['IMPACT_DAY_FLG']!=0,'IMPACT_DAY_FLG']=1

In [16]:
df_noLocation.loc[df_noLocation['PROVINCIAL_HOLIDAY_IND']!=0,'PROVINCIAL_HOLIDAY_IND']=1

In [17]:
df_noLocation.loc[df_noLocation['NATIONAL_HOLIDAY_IND']!=0,'NATIONAL_HOLIDAY_IND']=1

In [18]:
df_noLocation['DAY_OF_WEEK'].unique()

array([129, 172, 215, 258, 301,  43,  86], dtype=int64)

In [19]:
daylist = df_noLocation['DAY_OF_WEEK'].unique().tolist()

In [20]:
for i in daylist:
    df_noLocation.loc[df_noLocation['DAY_OF_WEEK']==i,'DAY_OF_WEEK']=i/43

In [21]:
df_noLocation['DAY_OF_WEEK'].unique()

array([3, 4, 5, 6, 7, 1, 2], dtype=int64)

In [22]:
df_noLocation.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME
0,2019-01-01,1,0,0,3,1,8
1,2019-01-02,0,0,0,4,1,40804
2,2019-01-03,0,0,0,5,1,42189
3,2019-01-04,0,0,0,6,1,44384
4,2019-01-05,0,0,0,7,1,11


In [143]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855 entries, 0 to 1854
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   x                    1855 non-null   float64
 1   y                    1855 non-null   float64
 2   STATION_NAME         1855 non-null   object 
 3   TOTAL_PRECIPITATION  1855 non-null   float64
 4   SNOW_ON_GROUND       1855 non-null   int64  
 5   HEATING_DEGREE_DAYS  1855 non-null   float64
 6   LOCAL_DAY            1855 non-null   int64  
 7   LOCAL_MONTH          1855 non-null   int64  
 8   MAX_TEMPERATURE      1855 non-null   float64
 9   MEAN_TEMPERATURE     1855 non-null   float64
 10  ID                   1855 non-null   object 
 11  MIN_TEMPERATURE      1855 non-null   float64
 12  LOCAL_YEAR           1855 non-null   int64  
 13  CLIMATE_IDENTIFIER   1855 non-null   int64  
 14  LOCAL_DATE           1855 non-null   object 
 15  PROVINCE_CODE        1855 non-null   o

In [144]:
df.describe()

,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,WEEK_OF_YEAR,DAY_OF_MONTH,MONTH_NO,YEAR_NO,COST_CENTRE_ID,PEAK,ACTUAL_VOLUME
count,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000,48977.000000
mean,0.029851,0.002634,0.034241,3.996488,25.378402,15.628622,6.323968,2021.436348,348453.209302,0.186128,725.996733
std,0.170177,0.051254,0.181848,2.000895,15.445775,8.808255,3.528853,1.743974,279039.751475,0.389214,1200.102897
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,2019.000000,51004.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,2.000000,12.000000,8.000000,3.000000,2019.000000,51535.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,4.000000,25.000000,16.000000,6.000000,2022.000000,247227.000000,0.000000,34.000000
75%,0.000000,0.000000,0.000000,6.000000,39.000000,23.000000,9.000000,2023.000000,641596.000000,0.000000,1027.000000
max,1.000000,1.000000,1.000000,7.000000,53.000000,31.000000,12.000000,2024.000000,651672.000000,1.000000,12428.000000


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48977 entries, 0 to 48976
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   CALENDAR_DATE           48977 non-null  datetime64[ns]
 1   NATIONAL_HOLIDAY_IND    48977 non-null  int64         
 2   PROVINCIAL_HOLIDAY_IND  48977 non-null  int64         
 3   IMPACT_DAY_FLG          48977 non-null  int64         
 4   DAY_OF_WEEK             48977 non-null  int64         
 5   WEEK_OF_YEAR            48977 non-null  int64         
 6   DAY_OF_MONTH            48977 non-null  int64         
 7   MONTH_NO                48977 non-null  int64         
 8   YEAR_NO                 48977 non-null  int64         
 9   COST_CENTRE_ID          48977 non-null  int64         
 10  PEAK                    48977 non-null  int64         
 11  SITE_PROVINCE_CODE      48977 non-null  object        
 12  ACTUAL_VOLUME           48977 non-null  int64 

In [23]:
df_noLocation['CALENDAR_DATE'] = pd.to_datetime(df_noLocation['CALENDAR_DATE'])

In [24]:
weather_df['LOCAL_DATE'] = pd.to_datetime(weather_df['LOCAL_DATE'])

In [148]:
weather_df.tail()

,x,y,STATION_NAME,TOTAL_PRECIPITATION,SNOW_ON_GROUND,HEATING_DEGREE_DAYS,LOCAL_DAY,LOCAL_MONTH,MAX_TEMPERATURE,MEAN_TEMPERATURE,ID,MIN_TEMPERATURE,LOCAL_YEAR,CLIMATE_IDENTIFIER,LOCAL_DATE,PROVINCE_CODE,COOLING_DEGREE_DAYS,TOTAL_RAIN,TOTAL_SNOW
1850,-125.216111,48.786111,CAPE BEALE LIGHT,3.0,0,11.7,9,2,9.0,6.3,1031316.2024.2.9,3.5,2024,1031316,2024-02-09,BC,0.0,3.0,0
1851,-125.216111,48.786111,CAPE BEALE LIGHT,25.4,0,12.0,10,2,7.5,6.0,1031316.2024.2.10,4.5,2024,1031316,2024-02-10,BC,0.0,25.4,0
1852,-125.216111,48.786111,CAPE BEALE LIGHT,27.2,0,10.2,11,2,10.0,7.8,1031316.2024.2.11,5.5,2024,1031316,2024-02-11,BC,0.0,27.2,0
1853,-125.216111,48.786111,CAPE BEALE LIGHT,0.0,0,9.7,12,2,12.0,8.3,1031316.2024.2.12,4.5,2024,1031316,2024-02-12,BC,0.0,0.0,0
1854,-125.216111,48.786111,CAPE BEALE LIGHT,0.0,0,11.0,13,2,11.5,7.0,1031316.2024.2.13,2.5,2024,1031316,2024-02-13,BC,0.0,0.0,0


In [25]:
weather_df = weather_df.rename(columns={'LOCAL_DATE': 'CALENDAR_DATE'})

In [26]:
attendence_df_grouped = attendence_df_grouped.rename(columns={'Date': 'CALENDAR_DATE'})

In [151]:
df_noLocation.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME
0,2019-01-01,1,0,0,3,1,8
1,2019-01-02,0,0,0,4,1,40804
2,2019-01-03,0,0,0,5,1,42189
3,2019-01-04,0,0,0,6,1,44384
4,2019-01-05,0,0,0,7,1,11


In [152]:
sorted_df = df_noLocation.sort_values(by='CALENDAR_DATE', ascending=True)

In [34]:
sorted_df.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME
0,2019-01-01,1,0,0,3,1,8
1,2019-01-02,0,0,0,4,1,40804
2,2019-01-03,0,0,0,5,1,42189
3,2019-01-04,0,0,0,6,1,44384
4,2019-01-05,0,0,0,7,1,11


In [22]:
df.query("NATIONAL_HOLIDAY_IND == 1 or PROVINCIAL_HOLIDAY_IND == 1").shape[0]

1591

In [23]:
df.query("(NATIONAL_HOLIDAY_IND == 1 or PROVINCIAL_HOLIDAY_IND == 1) & ACTUAL_VOLUME > 0").shape[0]

158

In [24]:
df.query("ACTUAL_VOLUME > 0").shape[0]

31349

In [25]:
df.query("ACTUAL_VOLUME == 0").shape[0]

17628

In [26]:
df.query("(NATIONAL_HOLIDAY_IND == 1 or PROVINCIAL_HOLIDAY_IND == 1) & ACTUAL_VOLUME > 0").shape[0]/df.query("NATIONAL_HOLIDAY_IND == 1 or PROVINCIAL_HOLIDAY_IND == 1").shape[0]

0.09930861093651791

In [27]:
df.query("(NATIONAL_HOLIDAY_IND == 0 & PROVINCIAL_HOLIDAY_IND == 0) & ACTUAL_VOLUME > 0").shape[0]/df.query("NATIONAL_HOLIDAY_IND == 0 & PROVINCIAL_HOLIDAY_IND == 0").shape[0]

0.6582323893132993

In [27]:
merged_df = pd.merge(df_noLocation,weather_df , on = "CALENDAR_DATE", how = "left")

In [28]:
full_merge_df = pd.merge(merged_df,attendence_df_grouped,on="CALENDAR_DATE",how="left")

In [155]:
full_merge_df.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME,x,y,STATION_NAME,...,LOCAL_YEAR,CLIMATE_IDENTIFIER,PROVINCE_CODE,COOLING_DEGREE_DAYS,TOTAL_RAIN,TOTAL_SNOW,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences
0,2019-01-01,1,0,0,3,1,8,-125.216111,48.786111,CAPE BEALE LIGHT,...,2019.0,1031316.0,BC,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2019-01-02,0,0,0,4,1,40804,-125.216111,48.786111,CAPE BEALE LIGHT,...,2019.0,1031316.0,BC,0.0,99.6,0.0,8885.90,1195.23,647.96,296.59
2,2019-01-03,0,0,0,5,1,42189,-125.216111,48.786111,CAPE BEALE LIGHT,...,2019.0,1031316.0,BC,0.0,28.8,0.0,9656.08,1218.42,562.20,283.91
3,2019-01-04,0,0,0,6,1,44384,-125.216111,48.786111,CAPE BEALE LIGHT,...,2019.0,1031316.0,BC,0.0,5.0,0.0,8795.96,1269.42,578.67,277.13
4,2019-01-05,0,0,0,7,1,11,-125.216111,48.786111,CAPE BEALE LIGHT,...,2019.0,1031316.0,BC,0.0,19.8,0.0,4354.98,575.00,305.32,132.69


In [71]:
trimed_merge_df=full_merge_df.drop(columns=['x', 'y','STATION_NAME','PROVINCE_CODE','CLIMATE_IDENTIFIER'])

In [72]:
trimed_merge_df.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME,TOTAL_PRECIPITATION,SNOW_ON_GROUND,HEATING_DEGREE_DAYS,...,ID,MIN_TEMPERATURE,LOCAL_YEAR,COOLING_DEGREE_DAYS,TOTAL_RAIN,TOTAL_SNOW,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences
0,2019-01-01,1,0,0,3,1,8,0.0,0.0,12.7,...,1031316.2019.1.1,3.5,2019.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2019-01-02,0,0,0,4,1,40804,99.6,0.0,10.5,...,1031316.2019.1.2,4.5,2019.0,0.0,99.6,0.0,8885.90,1195.23,647.96,296.59
2,2019-01-03,0,0,0,5,1,42189,28.8,0.0,9.2,...,1031316.2019.1.3,6.5,2019.0,0.0,28.8,0.0,9656.08,1218.42,562.20,283.91
3,2019-01-04,0,0,0,6,1,44384,5.0,0.0,9.0,...,1031316.2019.1.4,7.5,2019.0,0.0,5.0,0.0,8795.96,1269.42,578.67,277.13
4,2019-01-05,0,0,0,7,1,11,19.8,0.0,11.0,...,1031316.2019.1.5,5.5,2019.0,0.0,19.8,0.0,4354.98,575.00,305.32,132.69


In [73]:
trimed_merge_df=trimed_merge_df.drop(columns=['ID'])

In [31]:
trimed_merge_df.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME,TOTAL_PRECIPITATION,SNOW_ON_GROUND,HEATING_DEGREE_DAYS,...,MEAN_TEMPERATURE,MIN_TEMPERATURE,LOCAL_YEAR,COOLING_DEGREE_DAYS,TOTAL_RAIN,TOTAL_SNOW,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences
0,2019-01-01,1,0,0,3,1,8,0.0,0.0,12.7,...,5.3,3.5,2019.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2019-01-02,0,0,0,4,1,40804,99.6,0.0,10.5,...,7.5,4.5,2019.0,0.0,99.6,0.0,8885.90,1195.23,647.96,296.59
2,2019-01-03,0,0,0,5,1,42189,28.8,0.0,9.2,...,8.8,6.5,2019.0,0.0,28.8,0.0,9656.08,1218.42,562.20,283.91
3,2019-01-04,0,0,0,6,1,44384,5.0,0.0,9.0,...,9.0,7.5,2019.0,0.0,5.0,0.0,8795.96,1269.42,578.67,277.13
4,2019-01-05,0,0,0,7,1,11,19.8,0.0,11.0,...,7.0,5.5,2019.0,0.0,19.8,0.0,4354.98,575.00,305.32,132.69


In [86]:
trimed_merge_df=trimed_merge_df.drop(columns=['LOCAL_MONTH'])

In [87]:
trimed_merge_df=trimed_merge_df.drop(columns=['STATION_NAME'])

KeyError: "['STATION_NAME'] not found in axis"

In [74]:
trimed_merge_df_final = trimed_merge_df[trimed_merge_df['CALENDAR_DATE'] != '2019-01-01']

In [33]:
trimed_merge_df_final.head()

,CALENDAR_DATE,NATIONAL_HOLIDAY_IND,PROVINCIAL_HOLIDAY_IND,IMPACT_DAY_FLG,DAY_OF_WEEK,PEAK,ACTUAL_VOLUME,TOTAL_PRECIPITATION,SNOW_ON_GROUND,HEATING_DEGREE_DAYS,...,MEAN_TEMPERATURE,MIN_TEMPERATURE,LOCAL_YEAR,COOLING_DEGREE_DAYS,TOTAL_RAIN,TOTAL_SNOW,Scheduled Hours,Planned Absences,Unplanned Absences,Unpaid Absences
1,2019-01-02,0,0,0,4,1,40804,99.6,0.0,10.5,...,7.5,4.5,2019.0,0.0,99.6,0.0,8885.90,1195.23,647.96,296.59
2,2019-01-03,0,0,0,5,1,42189,28.8,0.0,9.2,...,8.8,6.5,2019.0,0.0,28.8,0.0,9656.08,1218.42,562.20,283.91
3,2019-01-04,0,0,0,6,1,44384,5.0,0.0,9.0,...,9.0,7.5,2019.0,0.0,5.0,0.0,8795.96,1269.42,578.67,277.13
4,2019-01-05,0,0,0,7,1,11,19.8,0.0,11.0,...,7.0,5.5,2019.0,0.0,19.8,0.0,4354.98,575.00,305.32,132.69
5,2019-01-06,0,0,0,1,1,346,1.4,0.0,11.7,...,6.3,4.5,2019.0,0.0,1.4,0.0,6250.94,676.13,472.46,253.29


In [75]:
trimed_merge_df_final['HOLIDAY_IND'] = trimed_merge_df_final['NATIONAL_HOLIDAY_IND'] + trimed_merge_df_final['PROVINCIAL_HOLIDAY_IND']

C:\Users\thedo\AppData\Local\Temp\ipykernel_4532\4072623166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimed_merge_df_final['HOLIDAY_IND'] = trimed_merge_df_final['NATIONAL_HOLIDAY_IND'] + trimed_merge_df_final['PROVINCIAL_HOLIDAY_IND']


In [91]:
trimed_merge_df_final = trimed_merge_df_final.drop(columns=['NATIONAL_HOLIDAY_IND','PROVINCIAL_HOLIDAY_IND'])

In [92]:
trimed_merge_df_final = trimed_merge_df_final.drop(columns=['MAX_TEMPERATURE','MIN_TEMPERATURE'])

In [93]:
trimed_merge_df_final = trimed_merge_df_final.drop(columns=['TOTAL_RAIN','TOTAL_SNOW'])

In [163]:
trimed_merge_df_final_nodate = trimed_merge_df_final.drop(columns=['CALENDAR_DATE'])

In [110]:
trimed_merge_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109 entries, 1 to 1138
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CALENDAR_DATE         1109 non-null   float64
 1   NATIONAL_HOLIDAY_IND  1109 non-null   int64  
 2   IMPACT_DAY_FLG        1109 non-null   int64  
 3   DAY_OF_WEEK           1109 non-null   int64  
 4   PEAK                  1109 non-null   int64  
 5   ACTUAL_VOLUME         1109 non-null   int64  
 6   TOTAL_PRECIPITATION   1109 non-null   float64
 7   SNOW_ON_GROUND        1109 non-null   float64
 8   MEAN_TEMPERATURE      1109 non-null   float64
 9   TOTAL_SNOW            1109 non-null   float64
 10  Scheduled Hours       1109 non-null   float64
 11  Planned Absences      1109 non-null   float64
 12  Unplanned Absences    1109 non-null   float64
 13  Unpaid Absences       1109 non-null   float64
 14  HOLIDAY_IND           1109 non-null   int64  
dtypes: float64(9), int64(

In [235]:
trimed_merge_df_final_nodate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109 entries, 1 to 1138
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IMPACT_DAY_FLG       1109 non-null   int64  
 1   DAY_OF_WEEK          1109 non-null   int64  
 2   PEAK                 1109 non-null   int64  
 3   ACTUAL_VOLUME        1109 non-null   int64  
 4   TOTAL_PRECIPITATION  1109 non-null   float64
 5   SNOW_ON_GROUND       1109 non-null   float64
 6   HEATING_DEGREE_DAYS  1109 non-null   float64
 7   LOCAL_DAY            1109 non-null   float64
 8   LOCAL_MONTH          1109 non-null   float64
 9   MAX_TEMPERATURE      1109 non-null   float64
 10  MEAN_TEMPERATURE     1109 non-null   float64
 11  MIN_TEMPERATURE      1109 non-null   float64
 12  LOCAL_YEAR           1109 non-null   float64
 13  COOLING_DEGREE_DAYS  1109 non-null   float64
 14  TOTAL_RAIN           1109 non-null   float64
 15  Scheduled Hours      1109 non-null   f

In [168]:
trimed_merge_df_final_nodate= trimed_merge_df_final_nodate.dropna()

In [234]:
trimed_merge_df_final_nodate = trimed_merge_df_final_nodate.drop(columns=['TOTAL_SNOW'])

In [77]:
trimed_merge_df_final= trimed_merge_df_final.dropna()

In [79]:
trimed_merge_df_final = trimed_merge_df_final.drop(columns=['PROVINCIAL_HOLIDAY_IND','TOTAL_RAIN','COOLING_DEGREE_DAYS','MAX_TEMPERATURE','MIN_TEMPERATURE','LOCAL_MONTH','LOCAL_YEAR','LOCAL_DAY','HEATING_DEGREE_DAYS',])

In [40]:
x = end_point - 90
last90_days = trimed_merge_df_final.loc[x:, :]
last90_days = last90_days['CALENDAR_DATE']

In [41]:
len(last90_days.to_numpy())

115

In [84]:
def date64_to_float(dt64):


    year = dt64.astype('M8[Y]')
    # print('year : ', year)
    days = (dt64 - year).astype('timedelta64[D]')
    # print('days:', days)
    year_next = year + np.timedelta64(1, 'Y')
    # print('year_next:', year_next)
    days_of_year = (year_next.astype('M8[D]') - year.astype('M8[D]')
                    ).astype('timedelta64[D]')
    # print('days_of_year:', days_of_year)
    date_float = 1970 + year.astype(float) + days / (days_of_year)
    # print('dt_float:', dt_float)
    return date_float

In [85]:
trimed_merge_df_final['CALENDAR_DATE'] = pd.to_datetime(trimed_merge_df_final['CALENDAR_DATE'], format='%Y-%m-%d')
trimed_merge_df_final['CALENDAR_DATE'] = date64_to_float(trimed_merge_df_final['CALENDAR_DATE'].to_numpy())

In [39]:
end_point = len(trimed_merge_df_final)

In [86]:
x = end_point - 90
trimed_merge_df_final_train = trimed_merge_df_final.loc[:x - 1, :]
trimed_merge_df_final_test = trimed_merge_df_final.loc[x:, :]

In [64]:
x = end_point - 90
trimed_merge_df_final_train_nodate = trimed_merge_df_final_nodate.loc[:x - 1, :]
trimed_merge_df_final_test_nodate = trimed_merge_df_final_nodate.loc[x:, :]

NameError: name 'trimed_merge_df_final_nodate' is not defined

In [312]:
trimed_merge_df_final_test_x = trimed_merge_df_final_test.loc[:, trimed_merge_df_final_test.columns != 'ACTUAL_VOLUME']
trimed_merge_df_final_test_y = trimed_merge_df_final_test['ACTUAL_VOLUME']
trimed_merge_df_final_train_x = trimed_merge_df_final_train.loc[:, trimed_merge_df_final_train.columns != 'ACTUAL_VOLUME']
trimed_merge_df_final_train_y = trimed_merge_df_final_train['ACTUAL_VOLUME']

In [380]:
trimed_merge_df_final_test_x = trimed_merge_df_final_test.loc[:, trimed_merge_df_final_test.columns != 'Scheduled Hours']
trimed_merge_df_final_test_y = trimed_merge_df_final_test['Scheduled Hours']
trimed_merge_df_final_train_x = trimed_merge_df_final_train.loc[:, trimed_merge_df_final_train.columns != 'Scheduled Hours']
trimed_merge_df_final_train_y = trimed_merge_df_final_train['Scheduled Hours']

In [153]:
trimed_merge_df_final_test_x = trimed_merge_df_final_test.loc[:, trimed_merge_df_final_test.columns != 'Planned Absences']
trimed_merge_df_final_test_y = trimed_merge_df_final_test['Planned Absences']
trimed_merge_df_final_train_x = trimed_merge_df_final_train.loc[:, trimed_merge_df_final_train.columns != 'Planned Absences']
trimed_merge_df_final_train_y = trimed_merge_df_final_train['Planned Absences']

In [87]:
trimed_merge_df_final_test_x = trimed_merge_df_final_test.loc[:, trimed_merge_df_final_test.columns != 'Unplanned Absences']
trimed_merge_df_final_test_y = trimed_merge_df_final_test['Unplanned Absences']
trimed_merge_df_final_train_x = trimed_merge_df_final_train.loc[:, trimed_merge_df_final_train.columns != 'Unplanned Absences']
trimed_merge_df_final_train_y = trimed_merge_df_final_train['Unplanned Absences']

In [240]:
trimed_merge_df_final_test_nodate_x = trimed_merge_df_final_test_nodate.loc[:, trimed_merge_df_final_test_nodate.columns != 'Unplanned Absences']
trimed_merge_df_final_test_nodate_y = trimed_merge_df_final_test_nodate['Unplanned Absences']
trimed_merge_df_final_train_nodate_x = trimed_merge_df_final_train_nodate.loc[:, trimed_merge_df_final_train_nodate.columns != 'Unplanned Absences']
trimed_merge_df_final_train_nodate_y = trimed_merge_df_final_train_nodate['Unplanned Absences']

In [102]:
scaler = MinMaxScaler()
scaler.fit(trimed_merge_df_final_train_x)

MinMaxScaler()

In [103]:
Xtrain = scaler.transform(trimed_merge_df_final_train_x)
Xtest = scaler.transform(trimed_merge_df_final_test_x)

In [524]:
reg = LinearRegression().fit(trimed_merge_df_final_train_x, trimed_merge_df_final_train_y)
lin_pre_y=reg.predict(trimed_merge_df_final_test_x)

In [525]:
mean_squared_error(trimed_merge_df_final_test_y, lin_pre_y)

26917.608000118165

In [104]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(80),
  tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=['mae'])
model.fit(Xtrain, trimed_merge_df_final_train_y, epochs=500, validation_data=(Xtrain, trimed_merge_df_final_train_y))

Epoch 1/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 542.9462 - mae: 543.0884 - val_loss: 214.7666 - val_mae: 219.6290
Epoch 2/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 202.6720 - mae: 202.3188 - val_loss: 170.6360 - val_mae: 169.7926
Epoch 3/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 147.9753 - mae: 148.1182 - val_loss: 131.0085 - val_mae: 130.8243
Epoch 4/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 131.6996 - mae: 131.6915 - val_loss: 135.7065 - val_mae: 134.3259
Epoch 5/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 129.9168 - mae: 129.7596 - val_loss: 143.6700 - val_mae: 142.1420
Epoch 6/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 131.9706 - mae: 132.1040 - val_loss: 136.8469 - val_mae: 135.5128
Epoch 7/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 125.0736 - mae: 125.0070 - val_loss: 122.2134 - val_mae: 123.5074
Epoch 8/500
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 124.1381 - mae: 124.1762 - val_loss: 119.7692 - val_mae: 120.5472


In [105]:
y_predict= model.predict(Xtest)
mean_absolute_error(trimed_merge_df_final_test_y, y_predict)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


109.74028011156166

In [64]:
hostory = model.fit(Xtrain, trimed_merge_df_final_train_y, epochs=50,
                   validation_data=(Xtest, trimed_merge_df_final_test_y))

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 51852032.0000 - val_loss: 1108873.2500
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1133573.1250 - val_loss: 277017.2812
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 342679.5000 - val_loss: 78094.2969
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 149008.6250 - val_loss: 43088.8828
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 96760.0781 - val_loss: 42907.9414
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 77534.5625 - val_loss: 49353.9727
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 75150.5078 - val_loss: 51790.7539
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 76441.6484 - val_loss: 54067.9531
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 65159.2773 - val_loss: 57245.5430
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 79183.2500 - val_loss: 57371.5000
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 72369.2969

In [65]:
y_predict= model.predict(Xtest)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [513]:
mean_absolute_error(trimed_merge_df_final_test_y, y_predict)

79.89991176970109

In [514]:
mean_squared_error(trimed_merge_df_final_test_y, y_predict)

15902.296863245258

In [106]:
len(y_predict)

115

In [507]:
y_predict = y_predict.flatten()

In [100]:
regr = RandomForestRegressor(n_estimators = 500,  max_depth=7, random_state=0)

In [101]:
regr.fit(trimed_merge_df_final_train_x, trimed_merge_df_final_train_y)

RandomForestRegressor(max_depth=7, n_estimators=500, random_state=0)

In [98]:
trimed_merge_df_final_pred_y = regr.predict(trimed_merge_df_final_test_x)

In [105]:
len(trimed_merge_df_final_pred_y)

115

In [112]:
output=pd.DataFrame({'Unplanned Absences':trimed_merge_df_final_pred_y,'date':last90_days}) 

In [113]:
output.to_csv(r'C:\Users\thedo\treeOutput.csv', encoding='utf-8', index=False,header=True)

In [111]:
mean_absolute_error(trimed_merge_df_final_test_y, trimed_merge_df_final_pred_y)

51.552568575907124

In [520]:
mean_squared_error(trimed_merge_df_final_test_y, trimed_merge_df_final_pred_y)

10918.312044146202